In [ ]:
import torch                                              # PyTorch 기본 라이브러리
import torch.nn as nn                                     # 신경망(Neural Network) 모델 구성을 위한 모듈
import torch.optim as optim                               # 최적화 알고리즘(Adam, SGD 등)을 위한 모듈
from torch.utils.data import DataLoader, random_split     # 데이터 로딩 및 데이터셋 분할을 위한 도구
from torchvision import datasets, transforms              # 비전(Vision) 데이터셋 및 이미지 변환 기능 제공

# -----------------------------
# 1) Device (장치 설정)
# -----------------------------
# Apple Silicon(M1/M2 등)의 MPS를 지원하면 MPS 사용, 아니면 CPU 사용
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print("device:", device)                                  # 현재 사용 중인 연산 기기 출력

# -----------------------------
# 2) Dataset / Transform (데이터셋 및 전처리)
#    - MNIST (28x28) -> 32x32 (양옆 2px 패딩 추가, LeNet 모델 입력 표준화)
#    - ToTensor() => 텐서 변환 및 픽셀 값을 [0, 1] 범위로 정규화
# -----------------------------
BATCH_SIZE = 128                                          # 학습 시 한 번에 모델에 넣을 데이터 개수
NUM_WORKERS = 0                                           # 데이터 로딩에 사용할 CPU 코어 수 (0은 메인 프로세스만 사용)

transform = transforms.Compose([
    transforms.Pad(2),                                    # 28x28 입력 이미지를 32x32 크기로 늘림 (Zero-padding)
    transforms.ToTensor(),                                # (H,W) 형태의 PIL 이미지를 (1,H,W) float32 텐서로 변환
])

# 학습용 데이터 다운로드 및 로드
full_train = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
# 테스트용 데이터 다운로드 및 로드
test_set   = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# 데이터셋을 학습용(90%)과 검증용(10%)으로 분할
train_size = int(len(full_train) * 0.9)                   # 전체 데이터의 90% 크기 설정
val_size   = len(full_train) - train_size                 # 나머지 10% 크기 설정
train_set, val_set = random_split(full_train, [train_size, val_size])

# 데이터 로더 설정 (모델에 배치 단위로 데이터를 공급)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=False)
val_loader   = DataLoader(val_set,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=False)
test_loader  = DataLoader(test_set,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=False)

# -----------------------------
# 3) LeNet 모델 구조 정의
#    특징 추출: Conv(6,5) -> Tanh -> AvgPool(2) -> Conv(16,5) -> Tanh -> AvgPool(2) -> Conv(120,5) -> Tanh
#    분류기: Flatten -> FC(84) -> Tanh -> FC(10)
# -----------------------------
class LeNet(nn.Module):
    def __init__(self, num_classes=10):                   # MNIST 클래스 개수는 총 10개 (0~9)
        super().__init__()                                # 부모 클래스(nn.Module)의 생성자 호출
        
        self.features = nn.Sequential(                    # 특징 추출부(CNN Layers)
            nn.Conv2d(1, 6, kernel_size=5),               # [1, 32, 32] -> [6, 28, 28] (크기 감소, 6개 채널 추출)
            nn.Tanh(),                                    # 활성화 함수 Tanh 통과
            nn.AvgPool2d(2),                              # [6, 28, 28] -> [6, 14, 14] (2x2 풀링으로 크기 반절 감소)

            nn.Conv2d(6, 16, kernel_size=5),              # [6, 14, 14] -> [16, 10, 10] (크기 감소, 16개 채널 추출)
            nn.Tanh(),                                    # 활성화 함수 Tanh 통과
            nn.AvgPool2d(2),                              # [16, 10, 10] -> [16, 5, 5] (2x2 풀링으로 크기 반절 감소)

            nn.Conv2d(16, 120, kernel_size=5),            # [16, 5, 5] -> [120, 1, 1] (크기 1x1 픽셀로 변환됨)
            nn.Tanh(),                                    # 활성화 함수 Tanh 통과
        )
        self.classifier = nn.Sequential(                  # 분류부(FC Layer, 완전연결 신경망)
            nn.Flatten(),                                 # 3차원 텐서(120*1*1)를 1차원(120)으로 평탄화
            nn.Linear(120, 84),                           # [120] -> [84] 완전 연결 계층
            nn.Tanh(),                                    # 활성화 함수 Tanh 통과
            nn.Linear(84, num_classes),                   # [84] -> [10] 출력 계층 (클래스 개수만큼 최종 예측값 계산)
        )

    def forward(self, x):                                 # 모델 연산 순서(순전파) 정의
        x = self.features(x)                              # 이미지가 특징 추출부를 통과
        x = self.classifier(x)                            # 추출된 특징이 최종적으로 분류부를 통과
        return x                                          # 최종 예측값(Logits) 반환

model = LeNet().to(device)                                # 생성한 모델 구조를 연산 기기(CPU/MPS) 메모리에 할당
print(model)                                              # 모델의 층과 구조 출력

# -----------------------------
# 4) Loss / Optimizer (손실 함수 및 최적화 함수)
# -----------------------------
criterion = nn.CrossEntropyLoss()                         # 다중 클래스 분류에 사용하는 교차 엔트로피 오차 손실 함수 (내부에 Softmax 포함)
optimizer = optim.Adam(model.parameters(), lr=1e-3)       # Adam 최적화 알고리즘 (학습률 0.001로 적용)

# -----------------------------
# 5) Train / Eval loops (학습 및 검증 루프)
# -----------------------------
def run_epoch(model, loader, train=True):
    if train:
        model.train()                                     # 모델을 학습 모드로 설정 (가중치 업데이트 가능 상태)
    else:
        model.eval()                                      # 모델을 평가 모드로 설정 (가중치 고정, Dropout 비활성화)

    total_loss, correct, total = 0.0, 0, 0                # 손실 누적, 맞춘 개수, 전체 개수 추적용 변수 0으로 초기화

    for x, y in loader:                                   # 데이터 로더에서 설정된 배치 크기(128개)만큼씩 반복해서 꺼냄
        x = x.to(device, non_blocking=True)               # 입력 데이터를 GPU(MPS) 또는 CPU 메모리로 이동
        y = y.to(device, non_blocking=True)               # 정답 데이터도 GPU(MPS) 또는 CPU 메모리로 이동

        if train:
            optimizer.zero_grad(set_to_none=True)         # 이전 학습의 기울기(gradient) 초기화 (새로운 배치를 위해서)

        with torch.set_grad_enabled(train):               # train=True 일 때만 메모리를 할당해 기울기(역전파 계산용) 추적
            logits = model(x)                             # 1. Forward Pass: 모델 시뮬레이션 및 예측값 얻기
            loss = criterion(logits, y)                   # 2. 오차 계산: 예측값(logits)과 정답(y) 사이의 차이 계산

            if train:
                loss.backward()                           # 3. Backward Pass: 오차를 뒤로 전달해서 기울기(역전파) 계산
                optimizer.step()                          # 4. 가중치 업데이트: 계산된 기울기에 맞추어 가중치 갱신

        total_loss += loss.item() * x.size(0)             # 각 배치의 오차를 현재 샘플 개수만큼 곱해 누적합
        pred = logits.argmax(dim=1)                       # 모델 예측 확률이 가장 높은 곳의 인덱스(클래스) 추출
        correct += (pred == y).sum().item()               # 모델 예측값과 실제 정답이 일치하는 개수 누적
        total += x.size(0)                                # 처리한 (배치) 데이터 개수 누적

    return total_loss / total, correct / total            # 1에포크 동안의 평균 손실값, 정확도 반환

EPOCHS = 10                                               # 총 학습 반복 횟수(에포크) 10회 설정
for epoch in range(1, EPOCHS + 1):                        # 1~10 에포크 동안 순서대로 반복
    tr_loss, tr_acc = run_epoch(model, train_loader, train=True)  # 학습용 데이터로 모델 학습 진행
    va_loss, va_acc = run_epoch(model, val_loader, train=False)   # 검증용 데이터로 모델 성능 확인 (옵티마이저 가동 안 함)
    
    # 1 에포크 종료 시마다 학습 결과를 터미널에 출력
    print(f"[{epoch}/{EPOCHS}] train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")

# -----------------------------
# 6) Test (최종 성능 평가)
# -----------------------------
te_loss, te_acc = run_epoch(model, test_loader, train=False)      # 마지막으로 처음 보는 테스트 세트로 실전 성능 평가
print("테스트 정확도:", te_acc)                           # 최종 정확도 확인


device: mps
LeNet(
  (features): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=120, out_features=84, bias=True)
    (2): Tanh()
    (3): Linear(in_features=84, out_features=10, bias=True)
  )
)
[1/10] train loss 0.0363 acc 0.9798 | val loss 0.0003 acc 0.9843
[2/10] train loss 0.0002 acc 0.9844 | val loss 0.0001 acc 0.9843
[3/10] train loss 0.0001 acc 0.9844 | val loss 0.0001 acc 0.9843
[4/10] train loss 0.0000 acc 0.9844 | val loss 0.0000 acc 0.9843
[5/10] train loss 0.0000 acc 0.9844 | val loss 0.0000 acc 0.9843
[6/10] train loss 0.0292 acc 0.9822 | val loss 0.0003 acc 0.9843
[7/10] train lo

In [ ]:
import torch                                              # PyTorch 기본 라이브러리
import torch.nn as nn                                     # 신경망(Neural Network) 모델 구성을 위한 모듈
import torch.optim as optim                               # 최적화 알고리즘(Adam, SGD 등)을 위한 모듈
from torch.utils.data import DataLoader                   # 데이터 로딩 및 배치를 위한 클래스
from torchvision import datasets, transforms              # 비전 데이터셋(CIFAR10 등) 및 이미지 변환 기능

# -----------------------------
# 1) Device (장치 설정)
# -----------------------------
device = torch.device("mps" if torch.mps.is_available() else "cpu") # Apple Silicon(MPS) 지원 시 사용, 아닐 경우 CPU 사용
print("device:", device)                                  # 현재 연산에 사용 중인 기기 출력

# -----------------------------
# 2) Dataset / Transform (데이터셋 및 전처리)
# -----------------------------
BATCH_SIZE = 128                                          # 학습/검증 시 한 번에 모델에 투입할 데이터 개수
NUM_WORKERS = 4                                           # 데이터 로딩용 CPU 코어 수 (환경에 따라 0~8 조절)
IMG_SIZE = 227                                            # AlexNet 구조에 맞춘 입력 이미지 크기 (227x227)

train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),              # 원본 이미지(32x32)를 AlexNet용 크기(227x227)로 조절
    transforms.ToTensor(),                                # 이미지를 Tensor로 변환하고 픽셀값을 [0, 1] 범위로 스케일링
])

test_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),              # 테스트 이미지도 동일하게 크기 조절
    transforms.ToTensor(),                                # Tensor 변환 및 스케일링
])

# 학습용/테스트용 CIFAR-10 데이터셋 다운로드 및 로드 (비행기, 자동차, 새, 고양이 등 10개 클래스)
train_set = datasets.CIFAR10(root="./data", train=True,  download=True, transform=train_tf)
test_set  = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tf)

# DataLoader를 통해 배치를 생성하고 병렬 처리 옵션 설정
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_set,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# -----------------------------
# 3) AlexNet (CIFAR-10) 모델 구조 정의
# -----------------------------
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):                   # CIFAR-10은 10개 클래스이므로 기본값 10
        super().__init__()                                # nn.Module의 초기화 메서드 호출
        
        self.features = nn.Sequential(                    # 특징 추출부(Convolutional Layers)
            nn.Conv2d(3, 96, kernel_size=11, stride=4),   # [3, 227, 227] -> [96, 55, 55] (스트라이드가 4로 커서 크기가 크게 감소)
            nn.ReLU(inplace=True),                        # 활성화 함수 ReLU 통과 (메모리 절약을 위해 inplace 사용)
            nn.MaxPool2d(kernel_size=3, stride=2),        # [96, 55, 55] -> [96, 27, 27] (최대 풀링으로 다운샘플링)

            nn.Conv2d(96, 256, kernel_size=5, padding=2), # [96, 27, 27] -> [256, 27, 27] (채널 수 증가, 패딩 2로 크기 유지)
            nn.ReLU(inplace=True),                        # 활성화 함수 ReLU 통과
            nn.MaxPool2d(kernel_size=3, stride=2),        # [256, 27, 27] -> [256, 13, 13]

            nn.Conv2d(256, 384, kernel_size=3, padding=1),# [256, 13, 13] -> [384, 13, 13]
            nn.ReLU(inplace=True),                        # 활성화 함수 ReLU 통과
            
            nn.Conv2d(384, 384, kernel_size=3, padding=1),# [384, 13, 13] -> [384, 13, 13] (채널, 크기 유지)
            nn.ReLU(inplace=True),                        # 활성화 함수 ReLU 통과
            
            nn.Conv2d(384, 256, kernel_size=3, padding=1),# [384, 13, 13] -> [256, 13, 13] (다시 채널 축소)
            nn.ReLU(inplace=True),                        # 활성화 함수 ReLU 통과
            nn.MaxPool2d(kernel_size=3, stride=2),        # [256, 13, 13] -> [256, 6, 6] (최종 특징 맵 공간)
        )
        
        self.classifier = nn.Sequential(                  # 분류부(Fully Connected Layers)
            nn.Flatten(),                                 # [256, 6, 6] 텐서를 1차원 벡터(크기 9216)로 평탄화
            nn.Linear(256 * 6 * 6, 4096),                 # [9216] -> [4096]
            nn.ReLU(inplace=True),                        # 활성화 함수 ReLU 통과
            nn.Dropout(0.5),                              # 과적합 방지를 위해 50%의 노드를 랜덤하게 Dropout으로 비활성화
            
            nn.Linear(4096, 4096),                        # [4096] -> [4096]
            nn.ReLU(inplace=True),                        # 활성화 함수 ReLU 통과
            nn.Dropout(0.5),                              # 한 번 더 과적합 방지 드롭아웃
            
            nn.Linear(4096, num_classes)                  # [4096] -> [10] 출력 계층 (클래스별 확률/로짓)
        )

    def forward(self, x):                                 # 모델의 순전파 연산 정의
        x = self.features(x)                              # 이미지가 특징 추출부 연산을 통과
        x = self.classifier(x)                            # 추출된 특징 벡터가 분류부 연산을 통과하여 결과 출력
        return x

model = AlexNet(num_classes=10).to(device)                # 생성된 모델을 연산 기기(CPU/MPS) 안으로 이동
print("params:", sum(p.numel() for p in model.parameters())/1e6, "M") # 모델의 전체 가중치 파라미터 수(백만 단위) 출력

# -----------------------------
# 4) Train / Eval (손실 함수 및 최적화, 학습 루프)
# -----------------------------
criterion = nn.CrossEntropyLoss()                         # 다중 분류용 오차 함수 세팅 (내부에 Softmax 함수가 포함됨)
optimizer = optim.Adam(model.parameters(), lr=1e-3)       # Adam 옵티마이저 설정 (초기 학습률 0.001)

def train_one_epoch(model, loader):                       # 1 에포크 학습 루프 함수
    model.train()                                         # 모델을 학습 모드로 설정 (Dropout 활성화)
    total_loss, correct, total = 0.0, 0, 0                # 누적 오차, 누적 맞춘 개수, 전체 관측 개수 초기화
    
    for x, y in loader:                                   # 데이터 로더에서 배치를 1개씩 꺼내옴
        x = x.to(device, non_blocking=True)               # 입력 이미지 데이터를 해당 장치(MPS 연산 등)로 넘김
        y = y.to(device, non_blocking=True)               # 라벨 데이터도 해당 장치로 넘김

        optimizer.zero_grad(set_to_none=True)             # 이전 배치의 역전파 기울기 값을 초기화 
        logits = model(x)                                 # 모델에 입력 데이터 넣어서 예측값 산출 (Forward Pass) 
        loss = criterion(logits, y)                       # 라벨과 예측값을 비교하여 오차(Loss) 계산
        loss.backward()                                   # 전체 Loss를 미분하여 역전파(Backward Pass)로 가중치 기울기를 추출
        optimizer.step()                                  # 추출된 기울기를 가지고 옵티마이저가 가중치를 조금씩 갱신

        total_loss += loss.item() * x.size(0)             # 각 배치의 오차를 현재 샘플 개수만큼 곱해 누적합
        pred = logits.argmax(dim=1)                       # 다항 분류 확률 중 가장 높은 확률일 때의 인덱스 산출
        correct += (pred == y).sum().item()               # 모델의 예측과 정답이 일치하는 경우를 찾아 맞춘 개수 누적
        total += x.size(0)                                # 처리한 전체 데이터 개수 카운팅
        
    return total_loss / total, correct / total            # 이번 에포크의 평균 오차와 평균 정확도 반환

@torch.no_grad()                                          # 이 함수 블록 내에서는 기울기를 계산/저장하지 않음 (속도 향상 및 메모리 절약)
def evaluate(model, loader):                              # 검증(평가) 루프 함수
    model.eval()                                          # 모델을 평가 모드로 설정 (가중치 변화 없고 Dropout 비활성화)
    total_loss, correct, total = 0.0, 0, 0                # 변수 초기화
    
    for x, y in loader:
        x = x.to(device, non_blocking=True)               # 장치 이동
        y = y.to(device, non_blocking=True)
        
        logits = model(x)                                 # 예측값 자체만 시뮬레이션함 (기울기 추적 안함)
        loss = criterion(logits, y)                       # 오차 점수 확인

        total_loss += loss.item() * x.size(0)             # 손실값 누적
        pred = logits.argmax(dim=1)                       # 가장 높은 확률 인덱스 산출
        correct += (pred == y).sum().item()               # 맞춘 예측 개수 누적 카운팅
        total += x.size(0)                                # 검증에 활용한 전체 표본치 누적
        
    return total_loss / total, correct / total            # 최종 평가 평균 오차와 정확도 반환

EPOCHS = 10                                               # 전체 반복 에포크(Epoch) 횟수 10으로 설정
for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader)# 한 에포크 동안 모델 학습 진행 후 학습 결과(오차, 정확도) 로드
    te_loss, te_acc = evaluate(model, test_loader)        # 학습 종료 직후 평가용 라벨링 데이터를 이용해 테스트 
    
    # 각 에포크마다 학습용 데이터와 검증(테스트) 데이터의 성능 변화를 터미널에 출력
    print(f"[{epoch}/{EPOCHS}] train loss {tr_loss:.4f} acc {tr_acc:.4f} | test loss {te_loss:.4f} acc {te_acc:.4f}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# -----------------------------
# 1) Device
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# -----------------------------
# 2) Dataset / Transform
#    - CIFAR10 (32x32) -> 224x224 resize
# -----------------------------
IMG_SIZE = 224
BATCH_SIZE = 128
NUM_WORKERS = 0  # Windows/Jupyter 안정

mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_set = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_tf)
test_set  = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tf)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=False)
test_loader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=False)

# -----------------------------
# 3) VGG16-like model (원형에 가깝게)
#    - Block 구성 동일
#    - Classifier: Flatten -> 4096 -> 4096 -> 10
# -----------------------------
class VGG16Like(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        def conv_block(in_c, out_c, n_conv):
            layers = []
            for i in range(n_conv):
                layers.append(nn.Conv2d(in_c if i == 0 else out_c, out_c, kernel_size=3, padding=1))
                layers.append(nn.ReLU(inplace=True))
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            return nn.Sequential(*layers)

        self.features = nn.Sequential(
            conv_block(3,   64, 2),  # Block1
            conv_block(64, 128, 2),  # Block2
            conv_block(128,256, 3),  # Block3
            conv_block(256,512, 3),  # Block4
            conv_block(512,512, 3),  # Block5
        )

        # 224 -> pool 5번 -> 224/32 = 7
        # feature map: 512 x 7 x 7
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = VGG16Like(num_classes=10).to(device)
print("params:", sum(p.numel() for p in model.parameters())/1e6, "M")

# -----------------------------
# 4) Loss / Optimizer
#    - sparse_categorical_crossentropy == CrossEntropyLoss
# -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# -----------------------------
# 5) Train / Eval loops
# -----------------------------
def train_one_epoch(model, loader):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for x, y in tqdm(loader):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)

    return total_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        logits = model(x)
        loss = criterion(logits, y)

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += x.size(0)

    return total_loss / total, correct / total

# -----------------------------
# 6) Train
# -----------------------------
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader)
    te_loss, te_acc = evaluate(model, test_loader)
    print(f"[{epoch}/{EPOCHS}] train loss {tr_loss:.4f} acc {tr_acc:.4f} | test loss {te_loss:.4f} acc {te_acc:.4f}")

print("테스트 정확도:", te_acc)